# Tracking the floating-point error

In the previous notebooks we were concerned with the convergence of the SCF procedure itself.
Now we change gears a little and take a look at analysing the error within an obtained SCF solution.

There are multiple sources of error in DFT (see [this paper](https://doi.org/10.1039/D0FD00048E) for details):
1. An obvious one is the DFT model itself
1. The error due to the discretisation (`Ecut` and `kgrid`)
1. The error due to aborting the SCF before getting the residual to zero (`tol`)
1. The error due to using a finite-precision floating-point arithmetic, the **floating-point error**

In this notebook we will focus on the floating-point error and to try and understand the floating-point error in a converged SCF solution. Or in other words:
$$ \text{How many digits of my solution are trustworthy?} $$

## Solving DFT with different floating-point types

One approximate way to get an idea of the floating-point error
in a procedure is to solve the problem using higher precision
floating-point types and compare the matching digits.

To make things a little more interesting we will use 32bit floating-point arithmetic:

In [ ]:
using DFTK

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# DFTK will use the floating-point type used to represent the lattice
# to deduce the floating-point type for the computation
model = model_DFT(Array{Float32}(lattice), atoms, [:lda_x, :lda_c_vwn])
basis = PlaneWaveBasis(model; Ecut=7, kgrid=[4, 4, 4], fft_size=(16, 16, 16))
scfres = self_consistent_field(basis, tol=1e-10)

results = Dict{DataType, Any}(
    Float32 => scfres.energies.total
)

@show results[Float32];

**Exercise:** To get a rough idea how many of these energy digits can be trusted,
re-run the computation using `Float64` and `Double64` (a floating point type offering even higher accuracy that 64bits). For the `Double64` case also converge the SCF to higher precision (e.g. `tol=1e-16`).
How many digits of the energy can be trusted at `Float32` and `Float64` level?

In [ ]:
using GenericLinearAlgebra  # Needed to enable generic fallbacks in DFTK
using DoubleFloats  # Defines Double64

# Your solution here

results[Float64]  = zero(Float64)
results[Double64] = zero(Double64)

@show results

## Using IntervalArithmetic to get a solid answer

Next we want to use the [`IntervalArithmetic`](https://github.com/JuliaIntervals/IntervalArithmetic.jl) package to rigorously track the error of the `Float32` computation.

The idea of interval arithmetic is to represent a number by two floats in form of an interval $[a, b]$. This interval is constructed in a way that the exact number is always inside the interval. Moreover this representation is tracked through the full flow of the program. I.e. all operations (addition, multiplication, more involved calls) are performed on both numbers $a$ and $b$ simultanously, but making sure that operations on $a$ always round down and those on $b$ always round up.
The result is thus again obtained as an interval,
which is moreover guaranteed to bracket the exact answer. As a result: The tighter the interval, the smaller the floating-point error.

To apply this to `DFTK` we first do the `Float32` computation as normal:

In [ ]:
using DFTK

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

model = model_DFT(Array{Float32}(lattice), atoms, [:lda_x, :lda_c_vwn], symmetries=false)
basis = PlaneWaveBasis(model; Ecut=7, kgrid=[4, 4, 4], fft_size=(16, 16, 16))
scfres = self_consistent_field(basis, tol=1e-10);

Next we check how far the resulting `scfres` is from being a fixed point.
Since iterative algorithms (e.g. diagonalisation) in interval arithmetic are tricky,
we will ensure $\rho = F(V(\rho))$ by checking
$$ H[\rho] \psi_i = \varepsilon_i \psi_i \qquad \text{for $\rho = \sum_i |\psi_i|^2$},$$
i.e. that the orbitals used for building a $\rho$ and from this the Hamiltonian $H[\rho]$
still diagonalise $H[\rho]$.
If this is the case this implies that a subsequent diagonalisation of the $H[\rho]$
can only get me the same $\rho$ back.

The floating-point error in this computation of this check we will fully track using `IntervalArithmetic`:

In [ ]:
using GenericLinearAlgebra
using IntervalArithmetic

# TODO Missing in IntervalArithmetic for Float32:
#      Interval ^ Interval not defined
#      Fixing PR: https://github.com/JuliaIntervals/IntervalArithmetic.jl/pull/482
import Base: ^
^(a::Interval{Float32}, x::Interval) = IntervalArithmetic.atomic(Interval{Float32}, IntervalArithmetic.bigequiv(a)^x)
# end 

# Get interval equivalents of key quantities
intModel = model_DFT(Array{Interval{Float32}}(lattice), atoms, [:lda_x, :lda_c_vwn], symmetries=false)
intBasis = PlaneWaveBasis(intModel; Ecut=7, kgrid=[4, 4, 4], fft_size=(16, 16, 16))
intOccupation = [Interval.(occk) for occk in scfres.occupation]
intEigvals = [Interval.(λk) for λk in scfres.eigenvalues]
intEigvecs = [Interval.(ψk) for ψk in scfres.ψ]

# Form density in interval arithmetic (i.e. evaluate F)
intρ = DFTK.compute_density(intBasis, intEigvecs, intOccupation)
@show maximum(radius, intρ)

# Compute energy and Hamiltionian in interval arithmetic using this density (i.e. evaluate V)
intEne, intHam = energy_hamiltonian(intBasis, intEigvecs, intOccupation;
                                    ρ=intρ, eigenvalues=intEigvals,
                                    εF=Interval(scfres.εF))
@show radius(intEne.total)

# Check the obtained eigenpairs are eigenpairs of this Hamiltonian
# (i.e. check we are at a fixed point)
residual_norm = similar(intEigvals)
for ik in 1:length(intBasis.kpoints)
    # Form Ritz values via Rayleigh quotient in interval arithmetic
    Λks = intEigvecs[ik]' * (intHam.blocks[ik] * intEigvecs[ik])
    residual_norm[ik] = norm.(eachcol(Λks - Diagonal(intEigvals[ik])))
end

n_converged = length(intEigvals[1]) - 3  # SCF contains 3 bands, which are not converged.
@show maximum(maximum(mid, knorm[1:n_converged]) for knorm in residual_norm)
@show maximum(maximum(radius, knorm[1:n_converged]) for knorm in residual_norm)
nothing

Clearly this calculation looses quite a bit of precision ...
Unfortunately IntervalArithmetic only guarantees that the first digit of the energy
can be trusted. Moreover it is not at all guaranteed that calculation has even converged!

However, one should notice that interval arithmetic in general overestimates the floating-point error. For example in this case one can already see that the density computation already looses about 3-4 significant digits. If we improve upon this (e.g. compute the density in `Float64`), then we still have a reasonable number of trustworthy digits in the energy (about 3-4).

#### Takeaway
- Trustworthy DFT calculations in pure `Float32` are tricky.
- IntervalArithmetic allows to identify routines where much precision is lost.
- If in doubt DFTK allows you to check your DFT calculation in higher precision (e.g. `Double64`) ... but of course that comes at a cost.